# Pre-training ALBERT

Code referred from
https://github.com/google-research/albert

In [ ]:
#Setting up TPU
%tensorflow_version 1.x
import os
import pprint
import json
import tensorflow as tf

assert "COLAB_TPU_ADDR" in os.environ, "ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!"
TPU_ADDRESS = "grpc://" + os.environ["COLAB_TPU_ADDR"] 
TPU_TOPOLOGY = "2x2"
print("TPU address is", TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
    # Now credentials are set for all future sessions on this TPU.

TensorFlow 1.x selected.
TPU address is grpc://10.15.73.98:8470
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 12787477634464250745),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 13264972788616172421),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 815571450894141198),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4663239132975362837),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 538891274723644301),
 _DeviceAttributes(/job:tpu_wo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Create sentencepiece file

In [ ]:
!pip install tensorflow==1.15.0

In [ ]:
# loading google drive to colaboratory
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# https://drive.google.com/file/d/1OkbkUULazQoS4m58PtdCYUyeQwRdLoJ1/view?usp=sharing
#Importing file to build vocab
id = '1OkbkUULazQoS4m58PtdCYUyeQwRdLoJ1'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('combineV2_book10per.txt') 

In [ ]:
#Cloning google research ALBERT github code
!pip install sentencepiece
!git clone https://github.com/google-research/ALBERT

     |████████████████████████████████| 1.1MB 2.7MB/s 
Cloning into 'ALBERT'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 356 (delta 0), reused 0 (delta 0), pack-reused 353
Receiving objects: 100% (356/356), 233.79 KiB | 3.00 MiB/s, done.
Resolving deltas: 100% (233/233), done.


In [ ]:
import sentencepiece as spm
spm.SentencePieceTrainer.Train('--input=''combineV2_book10per.txt'' --vocab_size=30000 --model_prefix=30k-clean --pad_id=0 --unk_id=1 --pad_piece=<pad> --unk_piece=<unk> --bos_id=-1 --eos_id=-1 --control_symbols=[CLS],[SEP],[MASK],[UNK],<pad> --user_defined_symbols="(,),",-,.,–,£,€"')

In [ ]:
sp_user = spm.SentencePieceProcessor()
sp_user.load('30k-clean.model')
vocabs = [sp_user.IdToPiece(id) for id in range(sp_user.GetPieceSize())]

In [ ]:
print(vocabs)

['<pad>', '<unk>', '[CLS]', '[SEP]', '[MASK]', '[UNK]', '(,),', '-', '.', '–', '£', '€"', '▁the', '▁to', '▁and', '▁a', '▁of', '▁i', '▁he', 'ed', 's', '▁was', '▁in', '▁it', '▁you', '▁his', '▁that', '▁her', '▁she', '▁is', '▁not', '▁with', '▁had', '▁as', '▁for', '▁on', 'ing', '▁but', '▁at', '▁my', '▁said', '▁me', '▁they', '▁him', '▁be', '▁have', '▁would', '▁from', '▁we', '▁what', '▁this', '▁out', '▁up', '▁were', '▁do', 'ly', '▁all', '▁one', '▁are', '▁so', '▁there', '▁could', '▁if', '▁back', '▁them', '▁like', '▁no', '▁into', 'd', '▁then', '▁did', '▁will', '▁just', '▁when', '▁about', '▁by', '▁their', '▁an', '▁your', '▁been', '▁or', '▁down', '▁know', '▁over', '▁time', '▁more', '▁now', 'y', '▁can', '▁who', '▁eyes', '▁see', '▁get', '▁how', '▁before', '▁only', '▁looked', '▁around', '▁way', '▁some', '▁here', '▁its', '▁head', '▁even', '▁through', '▁well', '▁man', '▁us', '▁am', '▁off', '▁other', '▁right', '▁than', '▁go', '▁hand', '▁still', '▁two', '▁away', '▁think', '▁again', '▁where', '▁something

In [ ]:
f = open("/content/drive/My Drive/Capstone/AlbertVocab/vocab.txt", "x")
f = open("/content/drive/My Drive/Capstone/AlbertVocab/vocab.txt", "w")
for vocab in vocabs:
  f.write(vocab)
  f.write('\n')

In [ ]:
sp_user.EncodeAsIds('Hello world')

[245, 1, 157, 10577, 232]

In [ ]:
!pip3 install albert-tensorflow

     |████████████████████████████████| 81kB 2.2MB/s 


In [ ]:
!pip install -r /content/ALBERT/requirements.txt

     |████████████████████████████████| 110.5MB 36kB/s 
     |████████████████████████████████| 92kB 7.3MB/s 
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0
  Found existing installation: tensorflow-hub 0.9.0
    Uninstalling tensorflow-hub-0.9.0:
      Successfully uninstalled tensorflow-hub-0.9.0


In [ ]:
cd ./ALBERT

/content/ALBERT


##Create Pre-training Data

In [ ]:
!pip install tensorflow-gpu==2.0.0-alpha0

import tensorflow as tf
import os
import sys

TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
WORKER_NAME='tpu_worker'

tf.config.experimental_connect_to_host(TPU_WORKER, WORKER_NAME) 

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(TPU_WORKER)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)
devices=tf.config.experimental_list_devices()
print(*devices,sep="\n")

     |████████████████████████████████| 332.1MB 53kB/s 
     |████████████████████████████████| 419kB 36.1MB/s 
     |████████████████████████████████| 3.0MB 44.5MB/s 


INFO:tensorflow:Initializing the TPU system: 10.15.73.98:8470
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Querying Tensorflow master (grpc://10.15.73.98:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 12787477634464250745)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 815571450894141198)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4663239132975362837)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 538891274723644301)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TP

In [ ]:
# !pip install tensorflow-gpu==1.15.0
import tensorflow as tf
import os
import sys

TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
WORKER_NAME='tpu_worker'

tf.config.experimental_connect_to_host(TPU_WORKER, WORKER_NAME) 

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(TPU_WORKER)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)
devices=tf.config.experimental_list_devices()
print(*devices,sep="\n")

INFO:tensorflow:Initializing the TPU system: 10.15.73.98:8470
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Querying Tensorflow master (grpc://10.15.73.98:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 12787477634464250745)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 815571450894141198)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4663239132975362837)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 538891274723644301)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TP

In [ ]:
with strategy.scope():
  !python create_pretraining_data.py --input_file "/content/combineV2_book10per.txt" --output_file "/content/drive/My Drive/Capstone/CreatePre/out_comb_v1" --vocab_file "/content/30k-clean.vocab" --spm_model_file "/content/30k-clean.model" --max_seq_length=256 --dupe_factor 1


W1030 12:48:04.059408 140316732458880 module_wrapper.py:139] From /usr/local/lib/python3.6/dist-packages/albert/tokenization.py:240: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:loading sentence piece model
I1030 12:48:04.059681 140316732458880 tokenization.py:240] loading sentence piece model
INFO:tensorflow:*** Reading from input files ***
I1030 12:48:04.179522 140316732458880 create_pretraining_data.py:631] *** Reading from input files ***
INFO:tensorflow:  /content/combineV2_book10per.txt
I1030 12:48:04.179761 140316732458880 create_pretraining_data.py:633]   /content/combineV2_book10per.txt

next
exit
quit
^C


In [ ]:
!pip install transformers
!pip install tfrecord

##Run Pre-training

In [ ]:
!python run_pretraining.py \
    --input_file="gs://capstonecs9-1/combinedPre" \
    --output_dir="gs://capstonecs9-1/models_recc2" \
    --albert_config_file="/content/drive/My Drive/Colab Notebooks/Capstone/albert_config.json" \
    --do_train \
    --do_eval \
    --train_batch_size=256 \
    --eval_batch_size=64 \
    --max_seq_length=512 \
    --max_predictions_per_seq=20 \
    --optimizer='lamb' \
    --learning_rate=.00176 \
    --num_train_steps=10000 \
    --num_warmup_steps=3025 \
    --use_tpu=True \
    --tpu_name='grpc://10.14.28.106:8470' \
    --save_checkpoints_steps=50



W1102 01:58:31.670285 140134263179136 module_wrapper.py:139] From /usr/local/lib/python3.6/dist-packages/albert/modeling.py:116: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

INFO:tensorflow:*** Input Files ***
I1102 01:58:32.952018 140134263179136 run_pretraining.py:484] *** Input Files ***
INFO:tensorflow:  gs://capstonecs9-1/combinedPre
I1102 01:58:32.952295 140134263179136 run_pretraining.py:486]   gs://capstonecs9-1/combinedPre
W1102 01:58:33.958949 140134263179136 estimator.py:1994] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f7326cca0d0>) includes params argument, but params are not passed to Estimator.
INFO:tensorflow:Using config: {'_model_dir': 'gs://capstonecs9-1/models_recc2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 50, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value

###Make Bin File

In [ ]:
import logging
import torch
# from transformers import AlbertConfig, AlbertModel
from transformers import AlbertConfig, AlbertForPreTraining, load_tf_weights_in_albert


logging.basicConfig(level=logging.INFO)


def convert_tf_checkpoint_to_pytorch(tf_checkpoint_path, albert_config_file, pytorch_dump_path):
    # Initialise PyTorch model
    config = AlbertConfig.from_json_file(albert_config_file)
    print("Building PyTorch model from configuration: {}".format(str(config)))
    model = AlbertForPreTraining(config)

    # Load weights from tf checkpoint
    load_tf_weights_in_albert(model, config, tf_checkpoint_path)

    # Save pytorch-model
    print("Save PyTorch model to {}".format(pytorch_dump_path))
    torch.save(model.state_dict(), pytorch_dump_path)



convert_tf_checkpoint_to_pytorch("/content/drive/My Drive/Reuters_Dataset/Reuters_model/model.ckpt-best.index","/content/drive/My Drive/Reuters_Dataset/Reuters_model/albert_config.json" , "/content/drive/My Drive/Reuters_Dataset/Reuters_model/model-fine/pytorch_model.bin")

Building PyTorch model from configuration: AlbertConfig {
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 20000
}

Save PyTorch model to /content/drive/My Drive/Reuters_Dataset/Reuters_model/model-fine/pytorch_model.bin
